In [1]:
import pandas as pd
import json
adj_df = pd.read_csv('grouped_movies_to_actors.csv')[['tconst', 'nconst']]

In [2]:
adj_df

,tconst,nconst
0,tt0061432,"['nm0396998', 'nm0398193']"
1,tt0062847,['nm0936259']
2,tt0062870,['nm0001428']
3,tt0062936,"['nm0398969', 'nm0001866', 'nm1787710', 'nm016..."
4,tt0062980,"['nm0820577', 'nm0281078', 'nm0715857', 'nm056..."
...,...,...
18686,tt9906644,"['nm2068971', 'nm7243877', 'nm1428724']"
18687,tt9907782,"['nm2933542', 'nm0677944']"
18688,tt9911006,"['nm9588822', 'nm1675134', 'nm3617629']"
18689,tt9914942,['nm10823679']


In [4]:
import numpy as np
thing = np.array([])
relations = []
for iter, row in adj_df.iterrows():
    inter = eval(row['nconst'])
    list2 = np.array(inter)
    thing = np.append(thing, list2)
    relations.append(inter)
print(len(relations))

18691


In [5]:
thing.size

49779

In [6]:
actors = np.unique(thing)
mapping = {k: v for v, k in enumerate(actors)}

In [26]:
N = len(mapping)
import scipy
A = scipy.sparse.lil_matrix((N, N), dtype=np.uint8)
from tqdm import tqdm
import itertools
counter = 0
for item in tqdm(relations):
        item = list(item)
        user = None
        if len(item) < 2:
            continue
        counter += 1
        if len(item) == 2:
            user = [item]
        else:
            user = list(itertools.combinations(item, 2))
        for pair in user:
            m1 = mapping[pair[0]]
            m2 = mapping[pair[1]]
            A[m1, m2] = 1
            A[m2, m1] = 1

100%|██████████| 18691/18691 [00:00<00:00, 28461.22it/s]


In [27]:
A = A.tocsr()
# Matrix Size
print((A.data.nbytes + A.indptr.nbytes + A.indices.nbytes) / 1e+6, "MB used")

0.57815 MB used


In [28]:
from scipy import sparse
sparse.save_npz("adjacency_matrix.npz", A)


In [29]:
import json
with open("mapping.json", "w") as fp:
    json.dump(mapping,fp) 